# Import Libraries

In [27]:
import pandas as pd
import numpy as np
import h5py
import spacy
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.model_selection import GridSearchCV, cross_val_score
from sklearn.ensemble import RandomForestClassifier
import matplotlib.pyplot as plt
from wordcloud import WordCloud
import os
import string as st
from transformers import BertTokenizer, TFBertModel
import tensorflow as tf
import torch
import fasttext
from sklearn.decomposition import PCA
from imblearn.ensemble import BalancedBaggingClassifier
from imblearn.combine import SMOTETomek
from imblearn.under_sampling import NearMiss
from sklearn.tree import DecisionTreeClassifier
from xgboost import XGBClassifier
from collections import Counter

pd.set_option('display.max_columns', None)  # Show all columns
pd.set_option('display.max_rows', None)  # Show all rows
pd.set_option('display.max_colwidth', None)  # Show full content in each cell
pd.set_option('display.width', 1000)  # Set max width

# Load spaCy's English model
nlp = spacy.load('en_core_web_sm')

# Linguistic Modules

In [56]:
def preprocess_text(text):
    # Define interrogative words to KEEP
    interrogatives = {"what", "why", "how", "who", "where", "when", "which", "whom", "whose"}
    custom_stopwords = set(nlp.Defaults.stop_words)
    custom_stopwords -= interrogatives

    doc = nlp(text.lower().strip())  # Lowercase and remove whitespace
    
# Process tokens: lemmatize, filter stopwords/punct/numbers, keep interrogatives
    tokens = [
        token.lemma_ 
        for token in doc 
        if (
            (not token.is_stop or token.text in interrogatives) and  # Keep interrogatives
            not token.is_punct and                                   # Remove punctuation
            (token.is_alpha or token.like_num)                       # Keep words/numbers
        )
    ]

    return ' '.join(tokens)

In [12]:
def extract_linguistic_features(text):
    doc = nlp(text)
    features = {
        'num_verbs': len([token for token in doc if token.pos_ == 'VERB']),
        'num_nouns': len([token for token in doc if token.pos_ == 'NOUN']),
        'sentence_length': len(doc),
        'blooms_verb_present': any(token.text in {'analyze', 'evaluate', 'create'} for token in doc)
    }
    return features

In [13]:
def get_sentence_vector(text):
    words = text.strip().split()
    vectors = [model.get_word_vector(w) for w in words]
    if not vectors:
        return np.zeros(300)
    return np.mean(vectors, axis=0)

In [ ]:
def create_word_filter(corpus, min_freq=10):
    """Creates a word frequency filter based on ACTUAL WORDS"""
    # Count WORD frequencies
    word_counter = Counter()

    for text in corpus:
        word_counter.update(text) 
    
    return {word for word, count in word_counter.items() if count >= min_freq}

def filter_text(text, word_filter):
    """Filters text using precomputed word filter"""
    return [word for word in text if word in word_filter]

In [ ]:
def get_vocabulary_size

# Pre-Processing

### Import Data

In [75]:
label_mapper = {
    'BT1' : 'knowledge',
    'BT2' : 'comprehension',
    'BT3' : 'application',
    'BT4' : 'analysis',
    'BT5' : 'synthesis',
    'BT6' : 'evaluation'
}

# Load dataset
df = pd.DataFrame()
for i in range(1,5):
    q_df = pd.read_csv(os.getcwd().replace('notebook' , 'dataset') + '/dataset' + str(i) + '.csv')
    df = pd.concat([df , q_df])

# Apply preprocessing
mask = df['label'].isin(label_mapper.keys())
df['label'] = df['label'].mask(mask, df['label'].map(label_mapper))
df['label'] = df['label'].str.lower()

df['processed_question'] = df['question'].apply(preprocess_text)

df['processed_question'] = [''.join(text) for text in df['processed_question']]

# Create frequency filter
min_frequency = 100
freq_filter = create_word_filter(df['processed_question'], min_frequency)

# Apply filter to all texts
filtered_corpus = [filter_text(text, freq_filter) for text in df['processed_question']]

# Convert back to strings
df['processed_question'] = [''.join(text) for text in filtered_corpus]

## Tokenize

BERT model is used to retain context

In [ ]:
# Load pre-trained BERT (TensorFlow version)
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = TFBertModel.from_pretrained('bert-base-uncased')

# Tokenize and encode
inputs = tokenizer(df['question'].tolist(), return_tensors='tf', padding=True, truncation=True)
outputs = model(**inputs)

# Extract embeddings (CLS token for sentence representation)
sentence_embedding = outputs.last_hidden_state[:, 0, :]
print("BERT Sentence Embedding Shape:", sentence_embedding.shape)

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions w

In [ ]:
pca = PCA(n_components=1000)
reduced_embeddings = pca.fit_transform(sentence_embedding.numpy())


### TF-IDF 
Execute either BERT or IF-IDF

In [76]:
vectorizer = TfidfVectorizer()
tfidf = vectorizer.fit_transform(df['processed_question'])

# Convert TF-IDF sparse matrix to DataFrame with appropriate column names
tfidf_df = pd.DataFrame(tfidf.toarray(), columns= vectorizer.get_feature_names_out(), index=df.index)
tfidf_df.shape

(10708, 7218)

Execute if linguistic features like count for verb,noun,etc needed

In [17]:
# Adding linguistic features
ling_features = df['processed_question'].apply(extract_linguistic_features).apply(pd.Series)

# Concatenate DataFrames
token_df = pd.concat([ling_features, tfidf_df], axis=1)

# Drop unnecessary columns (if needed)
token_df = token_df.drop(columns=['blooms_verb_present'], axis=1)
token_df.columns = token_df.columns.map(str)

# token_df = pca.fit_transform(token_df)

# Feature Extraction

### Label Balancing

In [24]:
x , y = tfidf_df , df['label']
x_resampled , y_resampled = SMOTETomek().fit_resample(x , y)

### Dimensionality Reduction

# Modelling

## Basic Machine Leanring

In [25]:
x_train , x_test , y_train, y_test = train_test_split(x_resampled, y_resampled, test_size=0.2, random_state=34 , stratify= y_resampled)

### Random Forest

In [26]:
params = {
    'n_estimators' : [2 , 5 , 10 , 20 , 50 , 100],
    'criterion' : ['gini' , 'entropy'],
    'max_depth' : [None , 5 , 10 , 20 , 50 , 100 , 200 , 350 , 500]
}
param_search = GridSearchCV(estimator= RandomForestClassifier(), param_grid= params, cv=5)

param_search.fit(x_resampled , y_resampled)

cv_results = pd.DataFrame(param_search.cv_results_)
cv_results = cv_results[["mean_test_score", "params"]].head(10).sort_values(by="mean_test_score", ascending=False)

cv_results 

,mean_test_score,params
5,0.816777,"{'criterion': 'gini', 'max_depth': None, 'n_estimators': 100}"
4,0.813710,"{'criterion': 'gini', 'max_depth': None, 'n_estimators': 50}"
3,0.805532,"{'criterion': 'gini', 'max_depth': None, 'n_estimators': 20}"
2,0.788515,"{'criterion': 'gini', 'max_depth': None, 'n_estimators': 10}"
1,0.765544,"{'criterion': 'gini', 'max_depth': None, 'n_estimators': 5}"
0,0.682562,"{'criterion': 'gini', 'max_depth': None, 'n_estimators': 2}"
9,0.462598,"{'criterion': 'gini', 'max_depth': 5, 'n_estimators': 20}"
8,0.386170,"{'criterion': 'gini', 'max_depth': 5, 'n_estimators': 10}"
7,0.312868,"{'criterion': 'gini', 'max_depth': 5, 'n_estimators': 5}"
6,0.251233,"{'criterion': 'gini', 'max_depth': 5, 'n_estimators': 2}"


In [17]:
classifier = RandomForestClassifier(n_estimators= 100 , max_depth= None , criterion= 'gini')
classifier.fit(x_train, y_train)
y_pred = classifier.predict(x_test)
print(classification_report(y_test , y_pred))

               precision    recall  f1-score   support

     analysis       0.90      0.88      0.89       564
  application       0.90      0.89      0.90       564
comprehension       0.84      0.83      0.83       554
   evaluation       0.81      0.78      0.79       559
    knowledge       0.74      0.80      0.77       547
    synthesis       0.79      0.80      0.79       558

     accuracy                           0.83      3346
    macro avg       0.83      0.83      0.83      3346
 weighted avg       0.83      0.83      0.83      3346



In [18]:
question = 'How many total disk access is needed to search a record using two level indexing?'
processed_q = preprocess_text(question)
tfidf_q = vectorizer.transform([processed_q])
tfidf_q = pd.DataFrame(tfidf_q.toarray(), columns=vectorizer.get_feature_names_out())



# Adding linguistic features
q_ling_features = pd.DataFrame([extract_linguistic_features(question)])

# Concatenate DataFrames
token_q = pd.concat([q_ling_features, tfidf_q], axis=1)
token_q = token_q.drop(columns= ['blooms_verb_present'])

print(classifier.predict(token_q))

['application']


In [ ]:
question = 'How many total disk access is needed to search a record using two level indexing?'
inputs =  tokenizer([question], return_tensors='tf', padding=True, truncation=True)
outputs = model(**inputs)

# Extract embeddings (CLS token for sentence representation)
sentence_embedding = outputs.last_hidden_state[:, 0, :]

### Decision Tree

In [ ]:
params = {
    'min_samples_split' : [2 , 5 , 10 , 20 , 50 , 100],
    'criterion' : ['gini' , 'entropy'],
    'max_depth' : [None , 5 , 10 , 20 , 50 , 100 , 200 , 350 , 500]
    # 'min_samples_leaf' : [1 ,2 , 5 , 10 , 20 , 50 , 100]
}
param_search = GridSearchCV(estimator= DecisionTreeClassifier(), param_grid= params, cv=5)

param_search.fit(x_resampled , y_resampled)

cv_results = pd.DataFrame(param_search.cv_results_)
cv_results = cv_results[["mean_test_score", "params"]].head(10).sort_values(by="mean_test_score", ascending=False)

cv_results 

,mean_test_score,params
54,0.733831,"{'criterion': 'entropy', 'max_depth': None, 'min_samples_split': 2}"
48,0.733293,"{'criterion': 'gini', 'max_depth': 500, 'min_samples_split': 2}"
90,0.733054,"{'criterion': 'entropy', 'max_depth': 200, 'min_samples_split': 2}"
102,0.732995,"{'criterion': 'entropy', 'max_depth': 500, 'min_samples_split': 2}"
42,0.732636,"{'criterion': 'gini', 'max_depth': 350, 'min_samples_split': 2}"
43,0.732337,"{'criterion': 'gini', 'max_depth': 350, 'min_samples_split': 5}"
0,0.731201,"{'criterion': 'gini', 'max_depth': None, 'min_samples_split': 2}"
36,0.730544,"{'criterion': 'gini', 'max_depth': 200, 'min_samples_split': 2}"
91,0.730245,"{'criterion': 'entropy', 'max_depth': 200, 'min_samples_split': 5}"
49,0.730006,"{'criterion': 'gini', 'max_depth': 500, 'min_samples_split': 5}"


In [31]:
dt = DecisionTreeClassifier(criterion= 'entropy', max_depth= None, min_samples_split = 2)
dt.fit(x_train, y_train)
y_pred = dt.predict(x_test)
print(classification_report(y_test , y_pred))

               precision    recall  f1-score   support

     analysis       0.80      0.83      0.82       564
  application       0.82      0.82      0.82       564
comprehension       0.78      0.79      0.78       554
   evaluation       0.74      0.75      0.74       559
    knowledge       0.71      0.70      0.70       547
    synthesis       0.73      0.69      0.71       558

     accuracy                           0.76      3346
    macro avg       0.76      0.76      0.76      3346
 weighted avg       0.76      0.76      0.76      3346



In [32]:
question = 'How many total disk access is needed to search a record using two level indexing?'
processed_q = preprocess_text(question)
tfidf_q = vectorizer.transform([processed_q])
tfidf_q = pd.DataFrame(tfidf_q.toarray(), columns=vectorizer.get_feature_names_out())



# Adding linguistic features
q_ling_features = pd.DataFrame([extract_linguistic_features(question)])

# Concatenate DataFrames
token_q = pd.concat([q_ling_features, tfidf_q], axis=1)
token_q = token_q.drop(columns= ['blooms_verb_present'])

print(dt.predict(token_q))

['comprehension']


### XGBoost

In [8]:
y_mapper = {
    'knowledge' : 0,
    'comprehension' : 1,
    'application' : 2,
    'analysis' : 3,
    'synthesis' : 4,
    'evaluation' : 5
}
xgb_y = y_resampled.map(y_mapper)

In [10]:
params = {
    'objective' : ['multi:softprob'],
    'max_depth' : [None , 5 , 10 , 50 , 100 , 200 , 500],
    'n_estimators': [2 , 5 , 10 , 50 , 100, 200],
    'learning_rate': [0.01, 0.1 , 0.5, 1]
    # 'subsample': [0.8, 0.9, 1.0]
}
param_search = GridSearchCV(estimator= XGBClassifier(), param_grid= params, cv=5)

param_search.fit(x_resampled , xgb_y)

cv_results = pd.DataFrame(param_search.cv_results_)
cv_results = cv_results[["mean_test_score", "params"]].head(10).sort_values(by="mean_test_score", ascending=False)

cv_results 

KeyboardInterrupt: 

### SVM